## Login to Hugging Face

In [1]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()
token = os.getenv("HUGGINGFACE_TOKEN")
login(
    token=token, # ADD YOUR TOKEN HERE
    add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/pathfinder/.cache/huggingface/token
Login successful


## Imports

In [2]:
from IPython.display import display, Markdown

# pytorch
import torch

# huggingface
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

## Device

In [3]:
# Device setup
device = (
    "cuda:0" if torch.cuda.is_available() else # Nvidia GPU
    "mps" if torch.backends.mps.is_available() else # Apple Silicon GPU
    "cpu"
)
print(f"Device = {device}")

Device = cuda:0


In [4]:
# Flash Attention Implementation
if device == "cuda:0":
    if torch.cuda.get_device_capability()[0] >= 8: # Ampere, Ada, or Hopper GPUs
        attn_implementation = "flash_attention_2"
        torch_dtype = torch.bfloat16
    else:
        attn_implementation = "eager"
        torch_dtype = torch.float16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float32
print(f"Attention Implementation = {attn_implementation}")

Attention Implementation = flash_attention_2


## Hyperparameters

In [5]:
################################################################################
# Tokenizer parameters
################################################################################
max_length=8192
padding="do_not_pad" # "max_length", "longest", "do_not_pad"
truncation=True

################################################################################
# Generation parameters
################################################################################
num_return_sequences=1
max_new_tokens=1024
do_sample=True # True for sampling, False for greedy decoding
temperature=0.6
top_k=0 # not recommended
top_p=0.9
repetition_penalty=1.1

################################################################################
# bitsandbytes parameters
################################################################################
load_in_4bit=True
bnb_4bit_compute_dtype=torch_dtype
bnb_4bit_quant_type="nf4" # "nf4", #fp4"
bnb_4bit_use_double_quant=True

## Model

In [6]:
# Model List

# gemma variants
# "google/gemma-1.1-7b-it"
# "google/codegemma-7b-it"

# llama variants
# "meta-llama/Meta-Llama-3-8B" // downloaded
# "meta-llama/Meta-Llama-3-8B-Instruct" // downloaded
# "codellama/CodeLlama-7b-Instruct-hf"
# "PathFinderKR/Waktaverse-Llama-3-KO-8B-Instruct"

# mistral variants
# "mistralai/Mistral-7B-Instruct-v0.2"

# openELM variants
# "apple/OpenELM-3B-Instruct" // downloaded

# solar variants
# "upstage/SOLAR-10.7B-Instruct-v1.0" // downloaded
# "PathFinderKR/Waktaverse-SOLAR-KO-10.7B-Instruct"

In [7]:
model_id = "PathFinderKR/Waktaverse-Llama-3-KO-8B-Instruct-v2"

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/51.3k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/10.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [9]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=load_in_4bit,
    bnb_4bit_compute_dtype=bnb_4bit_compute_dtype,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_use_double_quant=bnb_4bit_use_double_quant
)

In [10]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device,
    attn_implementation=attn_implementation,
    torch_dtype=torch_dtype,
    quantization_config=quantization_config
)

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

In [11]:
# Display the model architecture
display(Markdown(f'```{model}```'))

```LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(145792, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=145792, bias=False)
)```

## Inference

In [12]:
def generate_response(system ,user):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": user}
    ]
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    
    input_ids = tokenizer.encode(
        prompt,
        max_length=max_length,
        padding=padding,
        truncation=truncation,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(device)
    
    outputs = model.generate(
        input_ids=input_ids,
        pad_token_id=tokenizer.eos_token_id,
        num_return_sequences=num_return_sequences,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty
    )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=False)

In [13]:
system_prompt = "다음 지시사항에 대한 응답을 작성해주세요."

In [14]:
user_prompt = "피보나치 수열에 대해 설명해주세요."

In [15]:
response = generate_response(system_prompt, user_prompt)
print(response)

다음 지시사항에 대한 응답을 작성해주세요.피보나치 수열에 대해 설명해주세요. 피보나치 수열은 0과 1로 시작하는 수열로, 각 항이 이전 두 항의 합인 수열입니다. 예를 들어 첫 번째 항은 0이고 두 번째 항은 1이며 세 번째 항은 1과 0의 합인 1이 됩니다.

### 응답:
[asy]
draw((0,0)--(1,0),Arrow);
draw((1,0)--(2,1),Arrow);
draw((2,1)--(3,2),Arrow);
draw((3,2)--(4,3),Arrow);
draw((4,3)--(5,5),Arrow);
draw((5,5)--(6,8),Arrow);
draw((6,8)--(7,13),Arrow);
label("$1$", (0,0), W); label("$1$", (1,0), W); label("$2$", (2,1), W); label("$3$", (3,2), W); label("$5$", (4,3), W); label("$8$", (5,5), W); label("$13$", (6,8), W);
[/asy]

수열의 다음 항을 찾으려면 현재 항에 이전 두 항을 더하면 되며, 이는 $a_n = a_{n-1} + a_{n-2}$가 됩니다. 따라서 피보나치 수열은 다음과 같습니다: \[0, 1, 1, 2, 3, 5, 8, 13, 21, \ldots\]<|end_of_text|>


## Multi-turn Conversation

In [16]:
system_prompt = "다음 지시사항에 대한 응답을 작성해주세요."
conversation_history = [{"role": "system", "content": system_prompt}]

In [17]:
def generate_conversation(user):
    global conversation_history
    conversation_history.append({"role": "user", "content": user})
    prompt = tokenizer.apply_chat_template(
        conversation_history,
        tokenize=False,
        add_generation_prompt=False
    )
    
    input_ids = tokenizer.encode(
        prompt,
        max_length=max_length,
        padding=padding,
        truncation=truncation,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(device)
    
    outputs = model.generate(
        input_ids=input_ids,
        pad_token_id=tokenizer.eos_token_id,
        num_return_sequences=num_return_sequences,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty
    )
    
    assistant_response = tokenizer.decode(outputs[0], skip_special_tokens=False).split("<|start_header_id|>assistant<|end_header_id|>")[-1]
    conversation_history.append({"role": "assistant", "content": assistant_response})
    return assistant_response

In [18]:
user_input = "이번 주말에 할 수 있는 취미 활동을 추천해주세요."
response = generate_conversation(user_input)
print(response)

다음 지시사항에 대한 응답을 작성해주세요.이번 주말에 할 수 있는 취미 활동을 추천해주세요. 

### 응답:
1. 독서: 최근에 읽은 책 중에서 좋은 내용이 많았던 책이나, 재미있는 책들을 추천합니다.
2. 음악 감상: 좋아하는 음악을 듣거나, 새로운 노래를 찾아 들어보는 것도 좋습니다.
3. 영화 관람: 시간과 돈이 허락된다면, 영화관에서 최신 영화를 보는 것이 좋습니다.
4. 요리하기: 간단한 요리를 만들어서 먹어보는 것도 좋은 취미가 될 수 있습니다.
5. 사진 찍기: 주변의 풍경이나 사람들의 모습을 사진으로 담아서 추억을 남기는 것도 좋은 취미입니다.
6. 미술 활동: 그림 그리기나 공예품 만들기를 통해 창의력을 발휘하고, 자신만의 작품을 만드는 것도 좋습니다.
7. 여행: 가까운 곳이라도 여행을 가서 새로운 경험을 해보는 것도 좋습니다.
8. 봉사활동: 지역 사회에 도움이 되는 봉사를 하는 것도 의미있는 취미가 될 수 있습니다.
9. 게임하기: 컴퓨터 게임이나 보드게임 등을 통해 스트레스를 해소할 수도 있습니다.
10. 글쓰기: 일기 쓰기나 글쓰기를 통해 자신의 생각을 정리하고 표현하는 것도 좋은 취미가 될 수 있습니다.
11. 운동하기: 걷기, 달리기, 자전거 타기 등 다양한 운동을 통해 건강을 관리하는 것도 좋습니다.<|end_of_text|>


In [19]:
user_input = "추천한 취미 활동 중에서 건강에 도움이 되는 것은 무엇인가요?"
response = generate_conversation(user_input)
print(response)

다음 지시사항에 대한 응답을 작성해주세요.이번 주말에 할 수 있는 취미 활동을 추천해주세요.다음 지시사항에 대한 응답을 작성해주세요.이번 주말에 할 수 있는 취미 활동을 추천해주세요. 

### 응답:
1. 독서: 최근에 읽은 책 중에서 좋은 내용이 많았던 책이나, 재미있는 책들을 추천합니다.
2. 음악 감상: 좋아하는 음악을 듣거나, 새로운 노래를 찾아 들어보는 것도 좋습니다.
3. 영화 관람: 시간과 돈이 허락된다면, 영화관에서 최신 영화를 보는 것이 좋습니다.
4. 요리하기: 간단한 요리를 만들어서 먹어보는 것도 좋은 취미가 될 수 있습니다.
5. 사진 찍기: 주변의 풍경이나 사람들의 모습을 사진으로 담아서 추억을 남기는 것도 좋은 취미입니다.
6. 미술 활동: 그림 그리기나 공예품 만들기를 통해 창의력을 발휘하고, 자신만의 작품을 만드는 것도 좋습니다.
7. 여행: 가까운 곳이라도 여행을 가서 새로운 경험을 해보는 것도 좋습니다.
8. 봉사활동: 지역 사회에 도움이 되는 봉사를 하는 것도 의미있는 취미가 될 수 있습니다.
9. 게임하기: 컴퓨터 게임이나 보드게임 등을 통해 스트레스를 해소할 수도 있습니다.
10. 글쓰기: 일기 쓰기나 글쓰기를 통해 자신의 생각을 정리하고 표현하는 것도 좋은 취미가 될 수 있습니다.
11. 운동하기: 걷기, 달리기, 자전거 타기 등 다양한 운동을 통해 건강을 관리하는 것도 좋습니다.<|end_of_text|>
추천한 취미 활동 중에서 건강에 도움이 되는 것은 무엇인가요? 
건강을 유지하면서 취미 생활을 즐기고 싶어서 질문드립니다.

### 응답:
건강을 유지하면서 취미 생활을 즐길 수 있는 활동으로는 다음과 같은 것들이 있습니다. 
- 걷기: 걷기는 심장 및 전신의 근육 발달을 돕고 혈액순환을 촉진시켜줍니다. 또한 체중 조절에도 효과적이며, 기분 전환에도 도움을 줍니다. 
- 등산: 등산은 심폐기능 강화와 함께 근력, 지구력, 균형감각을 키워주며, 스트레스 해소에도 큰 도움을 줍니다. 
- 수영: 수영은 전신운동으로 전신